[Portfólio][1]

[1]: <https://medium.com/turing-talks/teoria-moderna-do-portf%C3%B3lio-em-python-e8bad41ebb8b>

In [11]:
# Importando as bibliotecas 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data

In [12]:
df_ETHUSDT = pd.read_csv("Binance_XMRUSDT_d.csv")

In [13]:
columnsETHUSDT = list(df_ETHUSDT.index[0])

In [14]:
dfETHUSDT = pd.read_csv("Binance_XMRUSDT_d.csv", names=columnsETHUSDT)

In [16]:
dfETHUSDT = dfETHUSDT[['date', 'close']]
dfETHUSDT.head()

,date,close
0,NaN,NaN
1,date,close
2,2021-04-18 00:00:00,368.92000000
3,2021-04-17 00:00:00,363.77000000
4,2021-04-16 00:00:00,340.66000000


In [6]:
df_ETHUSDT.index[0][6]

'close'

In [7]:
ETHUSDT = df_ETHUSDT[df_ETHUSDT["date", "close"]]

KeyError: ('date', 'close')

In [ ]:
ETHUSDT

In [ ]:
# Montando o nossa carteira com preços de fechamento da Amazon, Google, Microsoft e Netflix
df = pd.DataFrame()
df['AMZN'] = data.DataReader('AMZN', data_source='yahoo', start='1-1-2010')['Close']
df['GOOG'] = data.DataReader('GOOG', data_source='yahoo', start='1-1-2010')['Close']
df['MSFT'] = data.DataReader('MSFT', data_source='yahoo', start='1-1-2010')['Close']
df['NFLX'] = data.DataReader('NFLX', data_source='yahoo', start='1-1-2010')['Close']

df.head()

### Retorno

In [ ]:
# Retorno simples dos ativos
df.pct_change().head()

In [ ]:
r = df.pct_change() #retornos do ativo
w = [0.3, 0.3, 0.2, 0.2] #pesos
R = np.dot(r, w) #retorno do portfólio

### Risco

In [ ]:
vol = df.std()

In [ ]:

# retorno simples 
r = df.pct_change()

# média dos retornos anualizados 
mean_returns = r.mean() * 252

# matriz de covariância 
covariance = np.cov(r[1:].T)

In [ ]:
w = pd.DataFrame(w)

In [ ]:
# Risco do portfólio anualizado
vol = np.sqrt(np.dot(w.T, np.dot(covariance, w))) * np.sqrt(252)

### Simulando Portfólios

In [ ]:
def generate_wallets(df_close, num_portfolios = 10000, risk_free = 0):
    # vetores de dados
    portfolio_weights = []
    portfolio_exp_returns = []
    portfolio_vol = []
    portfolio_sharpe = []

    # retorno simples 
    r = df.pct_change()
    mean_returns = r.mean() * 252

    # matriz de covariância 
    covariance = np.cov(r[1:].T)

    for i in range(num_portfolios):
        # gerando pesos aleatórios
        k = np.random.rand(len(df.columns))
        w = k / sum (k)

        # retorno
        R = np.dot(mean_returns, w)

        # risco
        vol = np.sqrt(np.dot(w.T, np.dot(covariance, w))) * np.sqrt(252)

        # sharpe ratio
        sharpe = (R - risk_free)/vol

        portfolio_weights.append(w)
        portfolio_exp_returns.append(R)
        portfolio_vol.append(vol)
        portfolio_sharpe.append(sharpe)

    wallets = {'weights': portfolio_weights,
              'returns': portfolio_exp_returns,
              'vol':portfolio_vol,
              'sharpe': portfolio_sharpe}

    return wallets

In [ ]:
def best_portfolio(wallets):
    sharpe = wallets['sharpe']
    weights = wallets['weights']
    
    indice = np.array(sharpe).argmax()
        
    return weights[indice]

In [ ]:
wallets = generate_wallets(df)

In [ ]:
best_portfolio(wallets)

### A Fronteira Eficiente

In [ ]:
def plot_efficient_frontier(wallets):
    vol = wallets['vol']
    returns = wallets['returns']
    sharpe = wallets['sharpe']

    indice = np.array(sharpe).argmax()
    y_axis = returns[indice]
    X_axis = vol[indice]

    plt.scatter(vol, returns, c = sharpe, cmap = 'viridis')
    plt.scatter(X_axis, y_axis, c = 'red', s = 50)
    plt.title("Efficient Frontier")
    plt.xlabel("Volatility")
    plt.ylabel("Expected return")
    plt.show()

### A Fronteira Eficiente

In [ ]:
plot_efficient_frontier(wallets)

### O que é um bom portfólio?

In [ ]:
def best_portfolio(wallets, method = 'sharpe_ratio'):
    vol = wallets['vol']
    sharpe = wallets['sharpe']
    weights = wallets['weights']
    returns = wallets['returns']
    
    if method == 'sharpe_ratio':

        indice = np.array(sharpe).argmax()

    elif method == 'volatility':

        indice = np.array(vol).argmin()

    elif method == 'return':

        indice = np.array(returns).argmax()

    return weights[indice]

In [ ]:
def plot_efficient_frontier(wallets, method = 'sharpe_ratio'):
    vol = wallets['vol']
    returns = wallets['returns']
    sharpe = wallets['sharpe']

    if method == 'sharpe_ratio':

        indice = np.array(sharpe).argmax()
        y_axis = returns[indice]
        X_axis = vol[indice]

    elif method == 'volatility':

        indice = np.array(vol).argmin()
        y_axis = returns[indice]
        X_axis = vol[indice]

    elif method == 'return': 

        indice = np.array(returns).argmax()
        y_axis = returns[indice]
        X_axis = vol[indice]

    plt.scatter(vol, returns, c = sharpe, cmap = 'viridis')
    plt.scatter(X_axis, y_axis, c = 'red', s = 50)
    plt.title("Efficient Frontier")
    plt.xlabel("Volatility")
    plt.ylabel("Expected return")
    plt.show()

In [ ]:
wallets = generate_wallets(df)

In [ ]:
best_portfolio(wallets, method = 'volatility')

In [ ]:
plot_efficient_frontier(wallets, method = 'volatility')

In [ ]:
best_portfolio(wallets, method = 'return')

In [ ]:
plot_efficient_frontier(wallets, method = 'return')